<a href="https://colab.research.google.com/github/analyst-rhie/Kaggle/blob/main/World_War_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## World War 2
제2차 세계 대전의 공중 폭격 작전과 기상 조건의 다양한 데이터를 활용해보겠다.

순서는 다음과 같다.

1. 데이터 설명과 정리
2. 좀 더 이해하기 쉽도록 시각화

위 2가지 EDA과정의 프로세스를 먼저 진행 후 **폭격이 끝나는 시점을 예측**을 적으로 ARIMA 방법을 사용할 것이다.

* 시각화 라이브러리 중 캐글 코드나 구글 코랩을 사용하지 않고 자신의 컴(오프라인)에서 사용하려면 from plotly.offline import init_notebook_mode 를 import plotly.plotly as py 대신 사용하는데, Anaconda에서 별도로 설치해야 한다.  또한 이미 Plotly가 설치 됬다면 자주 업데이트 하는 것이 좋다.

```html
$ pip install plotly 
$ pip install plotly --upgrade
```

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
#import plotly.plotly as py # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go # plotly graphical object

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("/content/drive/MyDrive/Colab Notebooks/WorldWar2"))
# import warnings library
import warnings        
# ignore filters
warnings.filterwarnings("ignore") # if there is a warning after some codes, this will avoid us to see them.
plt.style.use('ggplot') # style of plots. ggplot is one of the most used style, I also like it.
# Any results you write to the current directory are saved as output.

['operations.csv', 'Summary of Weather.csv', 'Weather Station Locations.csv']


In [2]:
# bombing data
aerial = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/operations.csv")
aerial_temp = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/operations.csv")
# first weather data that includes locations like country, latitude and longitude.
weather_station_location = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/Weather Station Locations.csv")
weather_station_location_temp = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/Weather Station Locations.csv")

# Second weather data that includes measured min, max and mean temperatures
weather = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/Summary of Weather.csv")
weather_temp = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/WorldWar2/Summary of Weather.csv")

In [3]:
# drop countries that are NaN
aerial = aerial[pd.isna(aerial.Country)==False]
# drop if target longitude is NaN
aerial = aerial[pd.isna(aerial['Target Longitude'])==False]
# Drop if takeoff longitude is NaN
aerial = aerial[pd.isna(aerial['Takeoff Longitude'])==False]
# drop unused features
drop_list = ['Mission ID','Unit ID','Target ID','Altitude (Hundreds of Feet)','Airborne Aircraft',
             'Attacking Aircraft', 'Bombing Aircraft', 'Aircraft Returned',
             'Aircraft Failed', 'Aircraft Damaged', 'Aircraft Lost',
             'High Explosives', 'High Explosives Type','Mission Type',
             'High Explosives Weight (Pounds)', 'High Explosives Weight (Tons)',
             'Incendiary Devices', 'Incendiary Devices Type',
             'Incendiary Devices Weight (Pounds)',
             'Incendiary Devices Weight (Tons)', 'Fragmentation Devices',
             'Fragmentation Devices Type', 'Fragmentation Devices Weight (Pounds)',
             'Fragmentation Devices Weight (Tons)', 'Total Weight (Pounds)',
             'Total Weight (Tons)', 'Time Over Target', 'Bomb Damage Assessment','Source ID']
aerial.drop(drop_list, axis=1,inplace = True)
aerial = aerial[ aerial.iloc[:,8]!="4248"] # drop this takeoff latitude 
aerial = aerial[ aerial.iloc[:,9]!=1355]   # drop this takeoff longitude

In [8]:
aerial_temp.loc[pd.isna(aerial_temp['Target Longitude'])== False]

,Mission ID,Mission Date,Theater of Operations,Country,Air Force,Unit ID,Aircraft Series,Callsign,Mission Type,Takeoff Base,Takeoff Location,Takeoff Latitude,Takeoff Longitude,Target ID,Target Country,Target City,Target Type,Target Industry,Target Priority,Target Latitude,Target Longitude,Altitude (Hundreds of Feet),Airborne Aircraft,Attacking Aircraft,Bombing Aircraft,Aircraft Returned,Aircraft Failed,Aircraft Damaged,Aircraft Lost,High Explosives,High Explosives Type,High Explosives Weight (Pounds),High Explosives Weight (Tons),Incendiary Devices,Incendiary Devices Type,Incendiary Devices Weight (Pounds),Incendiary Devices Weight (Tons),Fragmentation Devices,Fragmentation Devices Type,Fragmentation Devices Weight (Pounds),Fragmentation Devices Weight (Tons),Total Weight (Pounds),Total Weight (Tons),Time Over Target,Bomb Damage Assessment,Source ID
0,1,8/15/1943,MTO,USA,12 AF,27 FBG/86 FBG,A36,NaN,NaN,PONTE OLIVO AIRFIELD,SICILY,37.131022,14.321464,40675,ITALY,SPADAFORA,NaN,NaN,NaN,38.220000,15.370000,NaN,20.0,NaN,20.0,NaN,NaN,NaN,NaN,40.000000,500 LB GP (GP-M43/M64),NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
1,2,8/15/1943,PTO,USA,5 AF,400 BS,B24,NaN,1,NaN,NaN,NaN,NaN,NaN,NEW GUINEA,KOMIATUM,RIDGE,NaN,1,-7.166667,147.000000,44.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,40.000000,1000 LB GP (GP-M44/M65),NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,9366.0
2,3,8/15/1943,MTO,USA,12 AF,27 FBG/86 FBG,A36,NaN,NaN,PONTE OLIVO AIRFIELD,SICILY,37.131022,14.321464,9630,ITALY,COSENZA,NaN,NaN,NaN,39.270000,16.250000,NaN,18.0,NaN,36.0,NaN,NaN,NaN,NaN,36.000000,500 LB GP (GP-M43/M64),NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN
3,4,8/15/1943,MTO,USA,12 AF,27 FBG/86 FBG,A36,NaN,NaN,PONTE OLIVO AIRFIELD,SICILY,37.131022,14.321464,16140,ITALY,GIOJA TAURO,NaN,NaN,NaN,38.430000,15.900000,NaN,15.0,NaN,15.0,NaN,1.0,NaN,NaN,30.000000,500 LB GP (GP-M43/M64),NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN
4,5,8/15/1943,PTO,USA,5 AF,321 BS,B24,NaN,1,NaN,NaN,NaN,NaN,NaN,SUMATRA,KILA,VILLAGE,NaN,1,-1.116667,103.883333,60.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,16.000000,1000 LB GP (GP-M44/M65),NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,22585.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178276,178738,2/16/1945,CBI,USA,10 AF,89 FS,P47,NaN,10,NaN,NaN,NaN,NaN,NaN,BURMA,NAMTOI,SUPPLY AREA,NaN,1,22.833333,96.916667,15.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,8.000000,250 LB GP (GP-M57),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,35815.0
178277,178739,2/16/1945,CBI,USA,10 AF,90 FS,P47,NaN,10,NaN,NaN,NaN,NaN,NaN,BURMA,MYITSON,UNIDENTIFIED TARGET,NaN,1,21.433333,96.466667,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,8.000000,250 LB GP (GP-M57),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,35020.0
178278,178740,2/16/1945,CBI,USA,10 AF,90 FS,P47,NaN,10,NaN,NaN,NaN,NaN,NaN,BURMA,MYITSON,UNIDENTIFIED TARGET,NaN,1,21.433333,96.466667,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,8.000000,250 LB GP (GP-M57),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,35022.0
178279,178741,2/16/1945,PTO,NEW ZEALAND,RNZAF,NO. 22 SQUADRON RNZAF,F4U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW IRELAND,KAVIENG SWEEP,VARIOUS TARGETS,NaN,NaN,-2.583333,150.833333,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,8.000000,1000 LB GP (GP-M44/M65),8000,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8000.0,4.0,NaN,NaN,NaN


In [11]:
aerial_temp = aerial_temp.loc[pd.isna(aerial_temp['Target Longitude'])== False]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169543 entries, 0 to 178280
Data columns (total 46 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Mission ID                             169543 non-null  int64  
 1   Mission Date                           169543 non-null  object 
 2   Theater of Operations                  169095 non-null  object 
 3   Country                                120521 non-null  object 
 4   Air Force                              120471 non-null  object 
 5   Unit ID                                50795 non-null   object 
 6   Aircraft Series                        169438 non-null  object 
 7   Callsign                               21 non-null      object 
 8   Mission Type                           47167 non-null   object 
 9   Takeoff Base                           3320 non-null    object 
 10  Takeoff Location                       3286 non-null    